## Available Serial Ports

In [ ]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
for port in ports:
    print(f"Port: {port.device}\tDescription: {port.description}")

## Read and Save 10000 Data points, and compute the sampling rate

In [ ]:
import serial
import matplotlib.pyplot as plt
import struct
import time
import os
import numpy as np

# Set up the serial port and the baud rate
ser = serial.Serial('COM3', 2000000)  
time.sleep(2)  # Wait for the serial connection to initialize
sample_count=10000
S1 = []
S2 = []

print("Reading data from Teensy...")
try:
    print("Starting data collection...")
    start_time = time.time()  # Start time for sampling rate calculation
    i = 0
    while i < sample_count:
        if ser.in_waiting > 7:  # 4 bytes each for two values (8 bytes)
            # Read data from the serial port
            data = ser.read(8)  # Read 8 bytes (2 integers)
            sensorValue1 = struct.unpack('I', data[:4])[0]  # Unpack as unsigned integer
            sensorValue2 = struct.unpack('I', data[4:])[0]  # Unpack as unsigned integer
            S1.append(sensorValue1)
            S2.append(sensorValue2)
            i += 1
    end_time = time.time()  # End time for sampling rate calculation
    elapsed_time = end_time - start_time  # Total time for samples
    average_sampling_rate = sample_count / elapsed_time  # Samples per second
    print(f"Average Sampling Rate: {average_sampling_rate:.2f} samples per second")
except KeyboardInterrupt:
    print("\nProgram interrupted.")
finally:
    ser.close()  # Close the serial port when done
    print("Serial connection closed.")

file_name = "100Hz"

# Specify the path to save the PDF inside the Data_Plots folder on the desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
plot_folder_path = os.path.join(desktop_path, "Data_Plots")

# Create the Data_Plots folder if it doesn't exist
os.makedirs(plot_folder_path, exist_ok=True)

# Save the plot as a PDF inside the Data_Plots folder
pdf_path = os.path.join(plot_folder_path, file_name + ".pdf")

# Save the numpy vectors as .npy files
npy_path_s1 = os.path.join(plot_folder_path, file_name + "_S1.npy")
npy_path_s2 = os.path.join(plot_folder_path, file_name + "_S2.npy")

np.save(npy_path_s1, S1)
np.save(npy_path_s2, S2)

plt.figure()
plt.plot(S1, label='Sensor 1')
plt.plot(S2, label='Sensor 2')
plt.xlabel('Time')
plt.ylabel('Sensor Value')
plt.title('Real-time Piezo Sensor Data')
plt.legend()
plt.savefig(pdf_path, format='pdf')
plt.show()


## Live reading and processing

In [ ]:
import serial
import collections
import struct
import time
import numpy as np
import mido
import scipy as sp
speed_of_sound = 5000  # Speed of sound in wood in m/s
distance_between_sensors=0.5 #Distance between the sensors in m
threshold=82 #least measured value before triggering a note
skip_measurements = False
measurements_count = 0
i=0
# Configure the serial connection to Teensy
window_size=100
average_sampling_rate =30456
# Initialize queues for sensor data
queue_sensor1 = collections.deque(maxlen=window_size)
queue_sensor2 = collections.deque(maxlen=window_size)

filename = "sensor_data_Pos1.txt"
data = np.loadtxt(filename, delimiter=',')
# Load data


def read_sensor_data(i):
    sensor1_data = data[i, 0]
    sensor2_data = data[i, 1]
    
    
    return sensor1_data, (sensor2_data)
def value_to_midi(value):
    if 1 <= value <= 7:
        note = 60 + value  # Mapping value to a MIDI note (Middle C is 60)
        velocity = 64  # Standard velocity
        return mido.Message('note_on', note=note, velocity=velocity)
    else:
        raise ValueError("Value must be between 1 and 7")

def distance_finder_corrFFT(sensor1,sensor2):
    
    correlation = sp.signal.correlate(queue_sensor1, queue_sensor2, mode='full')

    # Find the index of the maximum correlation
    max_index = np.argmax(correlation)

    delay = max_index /average_sampling_rate

    distance=delay*speed_of_sound +distance_between_sensors/2
    return int(distance)




def distance_finder_corrNP(sensor1,sensor2):
    
    correlation = np.correlate(queue_sensor1, queue_sensor2, mode='full')
    max_index = np.argmax(correlation)
    delay = max_index /average_sampling_rate

    distance=delay*speed_of_sound +distance_between_sensors/2
    return int(distance)
   

while (measurements_count<4000):
    i=0
    while(not skip_measurements):
        valueA14, valueA15 = read_sensor_data(i)
        if (valueA14>threshold) or (valueA15>threshold):
            skip_measurements=True
            while (i<window_size):
                valueA14, valueA15 = read_sensor_data(i)
                queue_sensor1.append(valueA14)
                queue_sensor2.append(valueA15)
                i+=1
                measurements_count+=1
            print(measurements_count)
            distance=distance_finder_corrNP(queue_sensor1,queue_sensor2)
            print(distance)
            print(f"Sensor1: {list(queue_sensor1)}")
            print(f"Sensor2: {list(queue_sensor2)}")
    i=0
    while(skip_measurements):
        skip_measurements=False
        while(i<window_size):
            valueA14, valueA15 = read_sensor_data(i)
            queue_sensor1.append(0)
            queue_sensor2.append(0)
            i+=1
        
        
        
        
    

measurements_count+=1